**Molecular Dynamics: NVE**

If we want to work at conditions beyond the athermal limit, we need to do dynamical simulations. Here, we use a technique called molecular dynamics, or MD.

As previously, lets setup a surface and run some examples. First up, loading modules and building a surface for Cu:

In [1]:
from ase.build import fcc111, add_adsorbate
# Set up a (4 x 4) two layer slab of Cu:
atoms = fcc111('Cu',size=(4,4,2))
atoms.set_pbc((1,1,0))

from ase import Atoms
# Add the N2 molecule oriented at 60 degrees:
d = 1.10 # N2 bond length
N2mol = Atoms('N2',positions=[[0.0,0.0,0.0],[0.5*3**0.5*d,0.5*d,0.0]])
add_adsorbate(atoms,N2mol,height=1.0,position='fcc')

Now as usual lets assign the calcutor to the object:

In [2]:
from ase.calculators.emt import EMT
atoms.set_calculator(EMT())

OK, now we need to think about our MD simulation. Let's initially run the calculatio at 300 K, using a Boltzmann distribution for the veolcities: 

In [3]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
# Set the momenta corresponding to T=300K
T = 300
MaxwellBoltzmannDistribution(atoms, T*units.kB)

Next, lets setup an NVE calculation (remembering that NVE is constant mols, volume and energy):

In [4]:
from ase.md.verlet import VelocityVerlet
dyn = VelocityVerlet(atoms, 5 * units.fs)  # 5 fs time step.

Now we are going to run the calculation. We are going to use a helper function to print energies - don't worry about the code structure, this is just to help us see the results as the simulation progresses:

In [5]:
def printenergy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

from ase.io.trajectory import Trajectory
# Now run the dynamics
dyn.attach(printenergy, interval=10)
traj = Trajectory('moldyn_nve.traj', 'w', atoms)
dyn.attach(traj.write, interval=10)
printenergy()

# Run for 200 steps, i.e. 200*5 fs = 1 ns
dyn.run(200)

Energy per atom: Epot = 0.425eV  Ekin = 0.037eV (T=286K)  Etot = 0.462eV
Energy per atom: Epot = 0.425eV  Ekin = 0.037eV (T=286K)  Etot = 0.462eV
Energy per atom: Epot = 0.366eV  Ekin = 0.091eV (T=705K)  Etot = 0.457eV
Energy per atom: Epot = 0.365eV  Ekin = 0.093eV (T=720K)  Etot = 0.458eV
Energy per atom: Epot = 0.382eV  Ekin = 0.076eV (T=589K)  Etot = 0.458eV
Energy per atom: Epot = 0.365eV  Ekin = 0.092eV (T=712K)  Etot = 0.457eV
Energy per atom: Epot = 0.362eV  Ekin = 0.095eV (T=736K)  Etot = 0.457eV
Energy per atom: Epot = 0.364eV  Ekin = 0.094eV (T=728K)  Etot = 0.458eV
Energy per atom: Epot = 0.394eV  Ekin = 0.065eV (T=502K)  Etot = 0.459eV
Energy per atom: Epot = 0.375eV  Ekin = 0.083eV (T=640K)  Etot = 0.457eV
Energy per atom: Epot = 0.381eV  Ekin = 0.076eV (T=587K)  Etot = 0.457eV
Energy per atom: Epot = 0.395eV  Ekin = 0.063eV (T=488K)  Etot = 0.458eV
Energy per atom: Epot = 0.382eV  Ekin = 0.076eV (T=587K)  Etot = 0.458eV
Energy per atom: Epot = 0.397eV  Ekin = 0.061eV (T=

True

Finally, once complete, lets visualise the system:

In [6]:
from ase.visualize import view
traj2 = Trajectory('moldyn_nve.traj')
view(traj2) #ngl doesn't seem to work here, can only run locally!

/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/io/jsonio.py:122: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  a = np.array(obj)


What is the behaviour of the system if you change the starting temperature, or the intial distribution of velocities? Alter the settings, rerun and see if your  data is as expected.

**Molecular Dynamics: NVT**

As our final calculation, lets run an NVT simulation and compare the results to our NVE.

We already have the settings and models loaded, so we can make use of this here and start our calculation from the end of the NVE simulation:

In [7]:
from ase.md.langevin import Langevin
dyn = Langevin(atoms, 5*units.fs, T*units.kB, 0.02)

All we need to do now the algorithm is setup is to run the calculation again:

In [8]:
# Now run the dynamics
dyn.attach(printenergy, interval=10)
traj = Trajectory('moldyn_nvt.traj', 'w', atoms)
dyn.attach(traj.write, interval=10)
printenergy()
dyn.run(200)

Energy per atom: Epot = 0.407eV  Ekin = 0.052eV (T=399K)  Etot = 0.458eV
Energy per atom: Epot = 0.407eV  Ekin = 0.052eV (T=399K)  Etot = 0.458eV
Energy per atom: Epot = 0.384eV  Ekin = 0.074eV (T=574K)  Etot = 0.458eV
Energy per atom: Epot = 0.392eV  Ekin = 0.054eV (T=419K)  Etot = 0.447eV
Energy per atom: Epot = 0.393eV  Ekin = 0.049eV (T=376K)  Etot = 0.441eV
Energy per atom: Epot = 0.373eV  Ekin = 0.069eV (T=534K)  Etot = 0.442eV
Energy per atom: Epot = 0.382eV  Ekin = 0.062eV (T=477K)  Etot = 0.443eV
Energy per atom: Epot = 0.387eV  Ekin = 0.057eV (T=443K)  Etot = 0.444eV
Energy per atom: Epot = 0.382eV  Ekin = 0.061eV (T=475K)  Etot = 0.443eV
Energy per atom: Epot = 0.377eV  Ekin = 0.062eV (T=477K)  Etot = 0.438eV
Energy per atom: Epot = 0.383eV  Ekin = 0.047eV (T=366K)  Etot = 0.430eV
Energy per atom: Epot = 0.379eV  Ekin = 0.046eV (T=354K)  Etot = 0.425eV
Energy per atom: Epot = 0.375eV  Ekin = 0.056eV (T=434K)  Etot = 0.431eV
Energy per atom: Epot = 0.395eV  Ekin = 0.038eV (T=

True

Notice how as the calculation runs the total energy decreases? Why is this?

Again, to visualise our results we can open the trajectory ( though not with `ngl`, it seems :( )

In [10]:
traj2 = Trajectory('moldyn_nvt.traj')
view(traj2)